In [2]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

from xgboost import XGBRegressor

import optuna

In [3]:
df_train = pd.read_csv('../input/30-days-of-ml/train.csv', index_col=0)
df_test = pd.read_csv('../input/30-days-of-ml/test.csv', index_col=0)

In [4]:
df_train.head()

## Model #1 

In [ ]:
df_train = pd.read_csv('../input/30-days-of-ml/train.csv', index_col=0)
df_test = pd.read_csv('../input/30-days-of-ml/test.csv', index_col=0)

X_train = df_train.drop('target', axis=1)
y_train = df_train.target
X_test = df_test

num_col = [col for col in X_train.columns if 'cat' not in col]
cat_col = [col for col in X_train.columns if 'cat' in col]

In [39]:
# Optimizing Using Optuna

def objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 7000),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.005,0.5),
        'max_depth': trial.suggest_int('max_depth', 1, 7),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 100.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 100.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'gamma': trial.suggest_float('gamma', 0.1, 1.0, step=0.1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
    }
    
    
    rmse=0
    kf = model_selection.KFold(n_splits=3, shuffle=True, random_state=42)

    for fold, (train_idx,valid_idx) in enumerate(kf.split(X_train)):
    
        # Generating X and y for train and test sets
        X_train_f = X_train.iloc[train_idx].copy()
        y_train_f = y_train.iloc[train_idx]

        X_valid_f = X_train.iloc[valid_idx].copy()
        y_valid_f = y_train.iloc[valid_idx]

        X_test = df_test.copy()

        # Encoding Categorical variables
        encoder = preprocessing.OrdinalEncoder()
        X_train_f[cat_col] = encoder.fit_transform(X_train_f[cat_col]) #cat_col
        X_valid_f[cat_col] = encoder.transform(X_valid_f[cat_col])
        X_test[cat_col] = encoder.transform(X_test[cat_col])

        # Scaling Features
        scaler = preprocessing.StandardScaler()
        X_train_f[num_col] = scaler.fit_transform(X_train_f[num_col])
        X_valid_f[num_col] = scaler.transform(X_valid_f[num_col])
        X_test[num_col] = scaler.transform(X_test[num_col])

        # Modeling 
        model = XGBRegressor(**params,
                             random_state=fold,
                             tree_method='gpu_hist',
                             gpu_id=0,
                             predictor='gpu_predictor')
    
        model.fit(X_train_f, y_train_f,
                  eval_set=[(X_valid_f,y_valid_f)],
                  early_stopping_rounds=300,
                  verbose=False)
        
        y_pred_f = model.predict(X_valid_f)
        rmse += metrics.mean_squared_error(y_pred_f, y_valid_f, squared=False)
        return rmse
    
    
# Optimizing
optuna.logging.set_verbosity(optuna.logging.WARNING) #stop showing each trial result

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

print(f'Best score: {study.best_value:.5f}')
print(f'Best Params: {study.best_params}')

In [23]:
# Final Model #1
y_test_pred = []

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx,valid_idx) in enumerate(kf.split(X_train)):
    
    # Generating X and y for train and test sets
    X_train_f = X_train.iloc[train_idx].copy()
    y_train_f = y_train.iloc[train_idx]
    
    X_valid_f = X_train.iloc[valid_idx].copy()
    y_valid_f = y_train.iloc[valid_idx]
    
    X_test = df_test.copy()
    
    # Encoding Categorical variables
    encoder = preprocessing.OrdinalEncoder()
    X_train_f[cat_col] = encoder.fit_transform(X_train_f[cat_col]) #cat_col
    X_valid_f[cat_col] = encoder.transform(X_valid_f[cat_col])
    X_test[cat_col] = encoder.transform(X_test[cat_col])
    
    # Scaling Features
    scaler = preprocessing.StandardScaler()
    X_train_f[num_col] = scaler.fit_transform(X_train_f[num_col])
    X_valid_f[num_col] = scaler.transform(X_valid_f[num_col])
    X_test[num_col] = scaler.transform(X_test[num_col])
    
    # Modeling
    model = XGBRegressor(**study.best_params,
                         random_state=fold,
                         tree_method='gpu_hist',
                         gpu_id=0,
                         predictor='gpu_predictor')
    model.fit(X_train_f, y_train_f)
    
    y_pred_f = model.predict(X_valid_f)
    rmse = metrics.mean_squared_error(y_pred_f, y_valid_f, squared=False)
    print(f'fold-{fold} rmse : {rmse:.5f}')
    
    y_test_f = model.predict(X_test)
    y_test_pred.append(y_test_f)

In [ ]:
df_train = pd.read_csv('../input/30-days-of-ml/train.csv', index_col=0)
df_test = pd.read_csv('../input/30-days-of-ml/test.csv', index_col=0)

X_train = df_train.drop('target', axis=1)
y_train = df_train.target
X_test = df_test

num_col = [col for col in X_train.columns if 'cat' not in col]
cat_col = [col for col in X_train.columns if 'cat' in col]

y_test_pred = []

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx,valid_idx) in enumerate(kf.split(X_train)):
    
    # Generating X and y for train and test sets
    X_train_f = X_train.iloc[train_idx].copy()
    y_train_f = y_train.iloc[train_idx]
    
    X_valid_f = X_train.iloc[valid_idx].copy()
    y_valid_f = y_train.iloc[valid_idx]
    
    X_test = df_test.copy()
    
    # Encoding Categorical variables
    encoder = preprocessing.OrdinalEncoder()
    X_train_f[cat_col] = encoder.fit_transform(X_train_f[cat_col]) #cat_col
    X_valid_f[cat_col] = encoder.transform(X_valid_f[cat_col])
    X_test[cat_col] = encoder.transform(X_test[cat_col])
    
    # Scaling Features
    scaler = preprocessing.StandardScaler()
    X_train_f[num_col] = scaler.fit_transform(X_train_f[num_col])
    X_valid_f[num_col] = scaler.transform(X_valid_f[num_col])
    X_test[num_col] = scaler.transform(X_test[num_col])
    
    # Modeling
    model = XGBRegressor(random_state=fold,
                         tree_method='gpu_hist',
                         gpu_id=0,
                         predictor='gpu_predictor')
    model.fit(X_train_f, y_train_f)
    
    y_pred_f = model.predict(X_valid_f)
    rmse = metrics.mean_squared_error(y_pred_f, y_valid_f, squared=False)
    print(f'fold-{fold} rmse : {rmse:.5f}')
    
    y_test_f = model.predict(X_test)
    y_test_pred.append(y_test_f)
    

In [ ]:
df_train = pd.read_csv('../input/30-days-of-ml/train.csv', index_col=0)
df_test = pd.read_csv('../input/30-days-of-ml/test.csv', index_col=0)

X_train = df_train.drop('target', axis=1)
y_train = df_train.target
X_test = df_test

num_col = [col for col in X_train.columns if 'cat' not in col]
cat_col = [col for col in X_train.columns if 'cat' in col]

# Log tranformation
for col in num_col:
    df_train[col] = np.log1p(df_train[col])
    df_test[col] = np.log1p(df_test[col])


kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx,valid_idx) in enumerate(kf.split(X_train)):
    
    # Generating X and y for train and test sets
    X_train_f = X_train.iloc[train_idx].copy()
    y_train_f = y_train.iloc[train_idx]
    
    X_valid_f = X_train.iloc[valid_idx].copy()
    y_valid_f = y_train.iloc[valid_idx]
    
    X_test = df_test.copy()
    
    # Encoding Categorical variables
    encoder = preprocessing.OrdinalEncoder()
    X_train_f[cat_col] = encoder.fit_transform(X_train_f[cat_col]) #cat_col
    X_valid_f[cat_col] = encoder.transform(X_valid_f[cat_col])
    X_test[cat_col] = encoder.transform(X_test[cat_col])
    
    # Modeling
    model = XGBRegressor(random_state=fold,
                         tree_method='gpu_hist',
                         gpu_id=0,
                         predictor='gpu_predictor')
    model.fit(X_train_f, y_train_f)
    
    y_pred_f = model.predict(X_valid_f)
    rmse = metrics.mean_squared_error(y_pred_f, y_valid_f, squared=False)
    print(f'fold-{fold} rmse : {rmse:.5f}')
    
    y_test_f = model.predict(X_test)
    y_test_pred.append(y_test_f)
    

In [ ]:
df_train = pd.read_csv('../input/30-days-of-ml/train.csv', index_col=0)
df_test = pd.read_csv('../input/30-days-of-ml/test.csv', index_col=0)

X_train = df_train.drop('target', axis=1).copy()
y_train = df_train.target
X_test = df_test

num_col = [col for col in X_train.columns if 'cat' not in col]
cat_col = [col for col in X_train.columns if 'cat' in col]

# Polynomials
poly = preprocessing.PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

train_poly = poly.fit_transform(X_train[num_col])
X_train_poly = pd.DataFrame(train_poly,
                            columns=[f'poly_{i}' for i in range(train_poly.shape[1])],
                            index=X_train.index) # using index for cancatenation
X_train = pd.concat([X_train[cat_col],X_train_poly], axis=1) # To avoid duplicating, we just concat cat_col with poly dataframe

test_poly = poly.fit_transform(df_test[num_col])
df_test_poly = pd.DataFrame(test_poly,
                            columns=[f'poly_{i}' for i in range(test_poly.shape[1])],
                            index=X_test.index)
df_test = pd.concat([X_test[cat_col],df_test_poly], axis=1)


# KFold
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx,valid_idx) in enumerate(kf.split(X_train)):
    
    # Generating X and y for train and test sets
    X_train_f = X_train.iloc[train_idx].copy()
    y_train_f = y_train.iloc[train_idx]
    
    X_valid_f = X_train.iloc[valid_idx].copy()
    y_valid_f = y_train.iloc[valid_idx]
    
    X_test = df_test.copy()
    
    # Encoding Categorical variables
    encoder = preprocessing.OrdinalEncoder()
    X_train_f[cat_col] = encoder.fit_transform(X_train_f[cat_col]) #cat_col
    X_valid_f[cat_col] = encoder.transform(X_valid_f[cat_col])
    X_test[cat_col] = encoder.transform(X_test[cat_col])
    
    # Modeling
    model = XGBRegressor(random_state=fold,
                         tree_method='gpu_hist',
                         gpu_id=0,
                         predictor='gpu_predictor')
    model.fit(X_train_f, y_train_f)
    
    y_pred_f = model.predict(X_valid_f)
    rmse = metrics.mean_squared_error(y_pred_f, y_valid_f, squared=False)
    print(f'fold-{fold} rmse : {rmse:.5f}')
    
    y_test_f = model.predict(X_test)
    y_test_pred.append(y_test_f)
    

In [ ]:
df_train = pd.read_csv('../input/30-days-of-ml/train.csv', index_col=0)
df_test = pd.read_csv('../input/30-days-of-ml/test.csv', index_col=0)

X_train = df_train.drop('target', axis=1)
y_train = df_train.target
X_test = df_test


num_col = [col for col in X_train.columns if 'cat' not in col]
cat_col = [col for col in X_train.columns if 'cat' in col]

y_test_pred = []

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx,valid_idx) in enumerate(kf.split(X_train)):

    # Generating X and y for train train_idxd test sets
    X_train_f = X_train.iloc[train_idx].copy()
    y_train_f = y_train.iloc[train_idx]
    
    X_valid_f = X_train.iloc[valid_idx].copy()
    y_valid_f = y_train.iloc[valid_idx]
    
    X_test = df_test.copy()
    
    df_train_f = pd.concat([X_train_f,y_train_f], axis=1)
    
    for col in cat_col:
        map_dict = df_train_f.groupby(col).mean().target.to_dict()
        X_train_f[col] = X_train_f[col].map(map_dict)
        X_valid_f[col] = X_valid_f[col].map(map_dict)
        X_test[col] = X_test[col].map(map_dict)

    # Modeling
    model = XGBRegressor(random_state=fold,
                         tree_method='gpu_hist',
                         gpu_id=0,
                         predictor='gpu_predictor')
    model.fit(X_train_f, y_train_f)
    
    y_pred_f = model.predict(X_valid_f)
    rmse = metrics.mean_squared_error(y_pred_f, y_valid_f, squared=False)
    print(f'fold-{fold} rmse : {rmse:.5f}')
    
    y_test_f = model.predict(X_test)
    y_test_pred.append(y_test_f)
    